<a href="https://colab.research.google.com/github/alandgmendes/excel_extraction_formulas_facilitadora/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl

In [28]:
import openpyxl
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Path to the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Fluxo de Caixa Diário.xlsx'

In [12]:
# Load the Excel workbook without data_only parameter
workbook = openpyxl.load_workbook(excel_file_path)

In [25]:
def create_dataframe_from_sheet(sheet):
    data = {'Coordinate': [], 'Value': []}
    for row in sheet.iter_rows():
        for cell in row:
            data['Coordinate'].append(cell.coordinate)
            data['Value'].append(str(cell.value))
    return pd.DataFrame(data)

def read_excel_and_create_dataframe(workbook):
    frames = []
    # Iterate over all sheets in the workbook
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Create DataFrame from the sheet
        df = create_dataframe_from_sheet(sheet)
        frames.append(df)

    # Concatenate DataFrames from all sheets
    result_df = pd.concat(frames, keys=workbook.sheetnames, names=['Sheet'])

    return result_df



In [26]:
# Output file path
output_file_path = 'cell_values.txt'


In [31]:
# Read cell values from the Excel file and create a DataFrame
df = read_excel_and_create_dataframe(workbook)
# Close the workbook
workbook.close()
# Export the DataFrame to a text file
#df.to_csv(output_file_path, sep='\t')

# Reset index to remove the 'Sheet' level
df_reset = df.reset_index(level='Sheet', drop=True)




In [35]:
# Create a new DataFrame with non-'None' values
filtered_df = df_reset[df_reset['Value'].str.lower() != 'none']


In [37]:
print(filtered_df.head(100))

    Coordinate                                              Value
0           A1                                               Menu
20          C2                                          Empresa: 
74          C5                                                1.0
76          E5                              Fluxo de Caixa Diário
94          E6  Registre diariamente as entradas e saídas do s...
..         ...                                                ...
233        AD7                                      =SUM(AD8:AD9)
234        AE7                                      =SUM(AE8:AE9)
235        AF7                                      =SUM(AF8:AF9)
236        AG7                                      =SUM(AG8:AG9)
237        AH7                                      =SUM(AH8:AH9)

[100 rows x 2 columns]


In [44]:
# Create a new DataFrame with formula values
formula_df = df[df['Value'].str.startswith('=')]



# Display the filtered DataFrame
print(formula_df.head(300))

                Coordinate            Value
Sheet                                      
FC-JANEIRO 1            B1     ='Início'!C2
           206          C7      =SUM(C8:C9)
           207          D7      =SUM(D8:D9)
           208          E7      =SUM(E8:E9)
           209          F7      =SUM(F8:F9)
...                    ...              ...
           2032       AA60  =SUM(AA61:AA68)
           2033       AB60  =SUM(AB61:AB68)
           2034       AC60  =SUM(AC61:AC68)
           2035       AD60  =SUM(AD61:AD68)
           2036       AE60  =SUM(AE61:AE68)

[300 rows x 2 columns]


In [45]:
# Get unique values in the 'Sheet' column
unique_sheet_values = df.index.get_level_values('Sheet').unique()

# Print the unique values
print(unique_sheet_values)

Index(['Início', 'FC-JANEIRO', 'FC-FEVEREIRO', 'FC-MARÇO', 'FC-ABRIL',
       'FC-MAIO', 'FC-JUNHO', 'FC-JULHO', 'FC-AGOSTO', 'FC-SETEMBRO',
       'FC-OUTUBRO', 'FC-NOVEMBRO', 'FC-DEZEMBRO', 'Fluxo de Caixa Mensal',
       'Fluxo de Caixa - Projetado', 'Fluxo de Caixa - Indicador', 'DRE',
       'DRE  - Projetado', 'DRE  - Indicador', 'Indicadores', 'Gráficos',
       'Plano de Ação'],
      dtype='object', name='Sheet')


In [46]:
# Reset the index to make 'Sheet' a regular column
df_reset = df.reset_index()

# Update the 'Sheet' column with the values from the previous index
df_reset['Sheet'] = df.index.get_level_values('Sheet')

# Print the updated DataFrame
print(df_reset)

                Sheet  level_1 Coordinate Value
0              Início        0         A1  Menu
1              Início        1         B1  None
2              Início        2         C1  None
3              Início        3         D1  None
4              Início        4         E1  None
...               ...      ...        ...   ...
669995  Plano de Ação    28995      Y1000  None
669996  Plano de Ação    28996      Z1000  None
669997  Plano de Ação    28997     AA1000  None
669998  Plano de Ação    28998     AB1000  None
669999  Plano de Ação    28999     AC1000  None

[670000 rows x 4 columns]


In [51]:
# Reset the index to make 'Sheet' a regular column
df = df.reset_index(level='Sheet')

# Print the updated DataFrame
print(df)

               Sheet Coordinate Value
0             Início         A1  Menu
1             Início         B1  None
2             Início         C1  None
3             Início         D1  None
4             Início         E1  None
...              ...        ...   ...
28995  Plano de Ação      Y1000  None
28996  Plano de Ação      Z1000  None
28997  Plano de Ação     AA1000  None
28998  Plano de Ação     AB1000  None
28999  Plano de Ação     AC1000  None

[670000 rows x 3 columns]


In [52]:
print(df["Sheet"].unique())

['Início' 'FC-JANEIRO' 'FC-FEVEREIRO' 'FC-MARÇO' 'FC-ABRIL' 'FC-MAIO'
 'FC-JUNHO' 'FC-JULHO' 'FC-AGOSTO' 'FC-SETEMBRO' 'FC-OUTUBRO'
 'FC-NOVEMBRO' 'FC-DEZEMBRO' 'Fluxo de Caixa Mensal'
 'Fluxo de Caixa - Projetado' 'Fluxo de Caixa - Indicador' 'DRE'
 'DRE  - Projetado' 'DRE  - Indicador' 'Indicadores' 'Gráficos'
 'Plano de Ação']


In [54]:
# Filter rows where 'Sheet' is equal to 'FC-JANEIRO' and 'Value' starts with '='
filtered_rows = df_reset[(df_reset['Sheet'] == 'FC-JANEIRO') & (df_reset['Value'].str.startswith('='))]

# Print the filtered DataFrame
print(filtered_rows)

           Sheet Coordinate         Value
1     FC-JANEIRO         B1  ='Início'!C2
206   FC-JANEIRO         C7   =SUM(C8:C9)
207   FC-JANEIRO         D7   =SUM(D8:D9)
208   FC-JANEIRO         E7   =SUM(E8:E9)
209   FC-JANEIRO         F7   =SUM(F8:F9)
...          ...        ...           ...
5809  FC-JANEIRO      AD171  =AD169+AD170
5810  FC-JANEIRO      AE171  =AE169+AE170
5811  FC-JANEIRO      AF171  =AF169+AF170
5812  FC-JANEIRO      AG171  =AG169+AG170
5813  FC-JANEIRO      AH171  =AH169+AH170

[785 rows x 3 columns]
